In [1]:
import sqlite3
from math import isnan

import pandas as pd

In [2]:
DATA_ROOT = "../exascale_data"
con = sqlite3.connect("test.db")

In [3]:
filename = "Ath_master_annotation_v01.txt"
path = f"{DATA_ROOT}/prerelease/node_tables/{filename}"
df = pd.read_csv(path, sep="\t")
df.tail()

,GID,defline,symbols,names,KO_effect,GO,GOdesc,mapman_code,mapman_name,mapman_desc
39814,AT4G09945,NaN,NaN,NaN,NaN,NaN,NaN,35.2,not assigned.unknown,long noncoding RNA
39815,AT4G09955,NaN,NaN,NaN,NaN,NaN,NaN,35.2,not assigned.unknown,Natural antisense transcript overlaps with AT4...
39816,AT4G09975,NaN,NaN,NaN,NaN,NaN,NaN,35.2,not assigned.unknown,Natural antisense transcript overlaps with AT4...
39817,AT4G09985,NaN,NaN,NaN,NaN,NaN,NaN,35.2,not assigned.unknown,Natural antisense transcript overlaps with AT4...
39818,AT4G09995,NaN,NaN,NaN,NaN,NaN,NaN,35.2,not assigned.unknown,long noncoding RNA


In [4]:
table_name = "nodes_ath"
df.to_sql(table_name, con, if_exists="replace")

In [5]:
pd.read_sql(f"""select * from "{table_name}" limit 1 """, con)

,index,GID,defline,symbols,names,KO_effect,GO,GOdesc,mapman_code,mapman_name,mapman_desc
0,0,AT1G01010,NAC domain containing protein 1,ANAC001 | NAC001 | NTL10,NAC domain containing protein 1 |,None,GO:0006355|GO:0000976|GO:0005634|GO:0003700,"regulation of transcription, DNA-templated|tra...",27.3.27,RNA.regulation of transcription.NAC domain tra...,NAC domain containing protein 1


In [6]:
#con.close()

In [7]:
isinstance(float('nan'), str) and float("asdf")

False

In [8]:
set([
    (len(go.split("|")) == len(godesc.split("|")))
    if isinstance(go, str) else go
    for go, godesc in df[["GO", "GOdesc"]].values
    if not (isinstance(go, float) and isnan(float(go)))
])

{True}

In [14]:
# How many nodes have a different number of symbols and names?
# This is much greater than 1, so they are probably not correlated.
len(set([
    (symbols, names)
    if (
            len(symbols.split("|")) != len(names.split("|"))
            if isinstance(symbols, str) else
            symbols
        ) else
    True
    for symbols, names in df[["symbols", "names"]].values
    if not (isinstance(names, float) and isnan(float(names)))
])) - 1 # ignore the ones with the same number of symbols and names

1974

In [10]:
%%time
godict = dict()
gotuples = sorted(sum([
    tuple(zip(go.split("|"), godesc.split("|")))
    for go, godesc in df[["GO", "GOdesc"]].values
    if not (isinstance(go, float) and isnan(float(go)))
], ()))
for go_, godesc_ in gotuples:
    if go_ not in godict:
        godict[go_] = godesc_
        continue
    assert godict[go_] == godesc_

CPU times: user 21.3 s, sys: 107 ms, total: 21.4 s
Wall time: 21.4 s


In [11]:
tuple(godict.items())[:5]

(('GO:0000002', 'mitochondrial genome maintenance'),
 ('GO:0000003', 'reproduction'),
 ('GO:0000009', 'alpha-1,6-mannosyltransferase activity'),
 ('GO:0000012', 'single strand break repair'),
 ('GO:0000014', 'single-stranded DNA endodeoxyribonuclease activity'))

In [12]:
isinstance(float('nan'), float)

True